<a href="https://colab.research.google.com/github/Responze31/Premier-League-Prediction/blob/main/footballPred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [1]:
import requests
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

API Setup

In [2]:
TOKEN = "a17d44d2afcb4ded8f2c90c9db9d59cc"
HEADERS = {"X-Auth-Token": TOKEN}
BASE = "https://api.football-data.org/v4"

def get_matches(season):
    url = f"{BASE}/competitions/PL/matches?season={season}"
    r = requests.get(url, headers=HEADERS)
    r.raise_for_status()
    return r.json()["matches"]

Build training dataset using 2024/25 season

In [3]:
data = []
for s in [2024]:
    for m in get_matches(s):
        if m["status"] == "FINISHED":
            data.append({
                "date": m["utcDate"],
                "home": m["homeTeam"]["name"],
                "away": m["awayTeam"]["name"],
                "hg": m["score"]["fullTime"]["home"],
                "ag": m["score"]["fullTime"]["away"]
            })

df = pd.DataFrame(data).sort_values("date").reset_index(drop=True)

Result encoding

In [4]:
def get_res(r):
    if r.hg > r.ag:
        return 0
    elif r.hg < r.ag:
        return 1
    else:
        return 2

df["res"] = df.apply(get_res, axis=1)

Enhanced team form with separate home/away tracking

In [5]:
def teams_form_enhanced(df, w=5):
    teams = pd.unique(df[["home", "away"]].values.ravel())
    home_hist = {t: [] for t in teams}
    away_hist = {t: [] for t in teams}
    rows = []

    for _, r in df.iterrows():
        h, a = r.home, r.away

        # Separate home and away form
        hf_home = home_hist[h][-w:]
        af_away = away_hist[a][-w:]

        rows.append({
            # Home team's HOME form
            "h_gf": np.mean([x[0] for x in hf_home]) if hf_home else 1.5,
            "h_ga": np.mean([x[1] for x in hf_home]) if hf_home else 1.0,
            # Away team's AWAY form
            "a_gf": np.mean([x[0] for x in af_away]) if af_away else 1.0,
            "a_ga": np.mean([x[1] for x in af_away]) if af_away else 1.5,
        })

        # Update separate home/away histories
        home_hist[h].append((r.hg, r.ag))
        away_hist[a].append((r.ag, r.hg))

    return pd.concat([df, pd.DataFrame(rows)], axis=1)

Calculate team strength ratings

In [6]:
def add_team_strength(df, historical_df):
    """Add team strength based on points per game and goal difference"""
    team_stats = {}

    for team in pd.unique(historical_df[["home", "away"]].values.ravel()):
        pts = 0
        gf = 0
        ga = 0
        games = 0

        # Home games
        home = historical_df[historical_df['home'] == team]
        for _, r in home.iterrows():
            games += 1
            gf += r.hg
            ga += r.ag
            if r.hg > r.ag:
                pts += 3
            elif r.hg == r.ag:
                pts += 1

        # Away games
        away = historical_df[historical_df['away'] == team]
        for _, r in away.iterrows():
            games += 1
            gf += r.ag
            ga += r.hg
            if r.ag > r.hg:
                pts += 3
            elif r.ag == r.hg:
                pts += 1

        ppg = pts / max(games, 1)
        gd_per_game = (gf - ga) / max(games, 1)

        team_stats[team] = {'ppg': ppg, 'gd': gd_per_game}

    df['h_strength'] = df['home'].map(lambda x: team_stats.get(x, {'ppg': 1.0})['ppg']).fillna(1.0)
    df['a_strength'] = df['away'].map(lambda x: team_stats.get(x, {'ppg': 1.0})['ppg']).fillna(1.0)
    df['h_gd'] = df['home'].map(lambda x: team_stats.get(x, {'gd': 0})['gd']).fillna(0)
    df['a_gd'] = df['away'].map(lambda x: team_stats.get(x, {'gd': 0})['gd']).fillna(0)

    return df

df = teams_form_enhanced(df)
df = add_team_strength(df, df)

Enhanced features

In [7]:
feat = ["h_gf", "h_ga", "a_gf", "a_ga", "h_strength", "a_strength", "h_gd", "a_gd"]
X = df[feat].values
y = df["res"].values

sc = StandardScaler()
X = sc.fit_transform(X)

Xtr, Xva, ytr, yva = train_test_split(X, y, test_size=0.2, random_state=42)

# Dataset
class MatchDS(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

Deeper model for better learning

In [8]:
class Net(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 3)
        )

    def forward(self, x):
        return self.net(x)

Train model with more epochs

In [9]:
train_dl = DataLoader(MatchDS(Xtr, ytr), batch_size=32, shuffle=True)
val_dl = DataLoader(MatchDS(Xva, yva), batch_size=256)

model = Net(X.shape[1])
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

print("Training model...")
for e in range(50):
    model.train()
    tot = 0
    for xb, yb in train_dl:
        opt.zero_grad()
        loss = loss_fn(model(xb), yb)
        loss.backward()
        opt.step()
        tot += loss.item()
    if (e+1) % 10 == 0:
        print(f"Epoch {e+1} | Loss {tot/len(train_dl):.4f}")

Training model...
Epoch 10 | Loss 0.9050
Epoch 20 | Loss 0.8536
Epoch 30 | Loss 0.8552
Epoch 40 | Loss 0.8018
Epoch 50 | Loss 0.7593


Evaluate

In [10]:
model.eval()
preds, true = [], []

with torch.no_grad():
    for xb, yb in val_dl:
        p = model(xb).argmax(1)
        preds.extend(p.numpy())
        true.extend(yb.numpy())

acc = accuracy_score(true, preds)
print(f"Validation Accuracy: {acc:.3f}")

Validation Accuracy: 0.526


Fetch 2025/26 matches

In [11]:
data_25 = []
for m in get_matches(2025):
    data_25.append({
        "date": m["utcDate"],
        "home": m["homeTeam"]["name"],
        "away": m["awayTeam"]["name"],
        "hg": m["score"]["fullTime"]["home"],
        "ag": m["score"]["fullTime"]["away"],
        "status": m["status"]
    })

df25 = pd.DataFrame(data_25).sort_values("date").reset_index(drop=True)
played = df25[df25.status == "FINISHED"].copy()
future = df25[df25.status == "TIMED"].copy()

print(f"\nPlayed: {len(played)} | Future: {len(future)}")


Played: 210 | Future: 170


Prepare features for future games

In [12]:
base_cols = ['date', 'home', 'away', 'hg', 'ag']
df_base = df[base_cols].copy()
played_base = played[base_cols].copy()

future_temp = future[['date', 'home', 'away']].copy()
future_temp['hg'] = 0
future_temp['ag'] = 0

all_games = pd.concat([df_base, played_base, future_temp], ignore_index=True)
all_games = teams_form_enhanced(all_games)

Add strength based on ALL historical data

In [13]:
all_historical = pd.concat([df[base_cols], played_base], ignore_index=True)
all_games = add_team_strength(all_games, all_historical)

future_feat = all_games.iloc[-len(future):][feat].values
future_feat = sc.transform(future_feat)

Predict with confidence boost

In [14]:
model.eval()
with torch.no_grad():
    probs = torch.softmax(
        model(torch.tensor(future_feat, dtype=torch.float32)), 1
    ).numpy()

make predictions more decisive

In [27]:
probs = probs ** 1.8
probs = probs / probs.sum(axis=1, keepdims=True)

future["p_home"] = probs[:,0]
future["p_away"] = probs[:,1]
future["p_draw"] = probs[:,2]

print("\n🔍 Prediction probabilities:")
print(f"Avg home: {future['p_home'].mean():.3f}")
print(f"Avg away: {future['p_away'].mean():.3f}")
print(f"Avg draw: {future['p_draw'].mean():.3f}")

arsenal_home = future[future['home'] == 'Arsenal FC']
if len(arsenal_home) > 0:
    print(f"Arsenal home avg: {arsenal_home['p_home'].mean():.3f}")


🔍 Prediction probabilities:
Avg home: 0.234
Avg away: 0.654
Avg draw: 0.113
Arsenal home avg: 0.356


Monte-Carlo simulation

In [28]:
teams = pd.unique(df25[["home","away"]].values.ravel())
points = {t:0 for t in teams}

for _, r in played.iterrows():
    if r.hg > r.ag:
        points[r.home] += 3
    elif r.hg < r.ag:
        points[r.away] += 3
    else:
        points[r.home] += 1
        points[r.away] += 1

def simulate():
    pts = points.copy()
    team_form = {t: 0 for t in teams}  # Track recent form

    for _, r in future.iterrows():
        probs = np.array([r.p_home, r.p_away, r.p_draw])
        probs = probs / probs.sum()

        # Add random noise
        noise = np.random.normal(0, 0.20, 3)

        # Adjust based on recent form (hot/cold streaks)
        form_boost_home = team_form[r.home] * 0.05  # 5% per recent win
        form_boost_away = team_form[r.away] * 0.05

        probs[0] += form_boost_home  # Home win probability
        probs[1] += form_boost_away  # Away win probability

        probs = probs + noise
        probs = np.maximum(probs, 0.05)
        probs = probs / probs.sum()

        res = np.random.choice([0, 1, 2], p=probs)

        # Update points
        if res == 0:
            pts[r.home] += 3
            team_form[r.home] = min(team_form[r.home] + 1, 3)  # Cap at +3
            team_form[r.away] = max(team_form[r.away] - 1, -3)
        elif res == 1:
            pts[r.away] += 3
            team_form[r.away] = min(team_form[r.away] + 1, 3)
            team_form[r.home] = max(team_form[r.home] - 1, -3)
        else:
            pts[r.home] += 1
            pts[r.away] += 1
            # Form slowly reverts to 0 on draws
            team_form[r.home] = team_form[r.home] * 0.5
            team_form[r.away] = team_form[r.away] * 0.5

    return pts

cumulative_points = {t: 0 for t in teams}
title_wins = {t: 0 for t in teams}

n_sims = 1000
print(f"\nRunning {n_sims} simulations...")
for i in range(n_sims):
    if i % 100 == 0:
        print(f"  {i}/{n_sims}...")

    sim_points = simulate()
    winner = max(sim_points, key=sim_points.get)
    title_wins[winner] += 1

    for team in teams:
        cumulative_points[team] += sim_points[team]

avg_points = {t: cumulative_points[t] / n_sims for t in teams}


Running 1000 simulations...
  0/1000...
  100/1000...
  200/1000...
  300/1000...
  400/1000...
  500/1000...
  600/1000...
  700/1000...
  800/1000...
  900/1000...


Calculate stats

In [29]:

team_stats = {}
for team in teams:
    team_stats[team] = {
        'played': 0, 'wins': 0, 'draws': 0, 'losses': 0,
        'gf': 0, 'ga': 0
    }

for _, r in played.iterrows():
    team_stats[r.home]['played'] += 1
    team_stats[r.home]['gf'] += r.hg
    team_stats[r.home]['ga'] += r.ag

    team_stats[r.away]['played'] += 1
    team_stats[r.away]['gf'] += r.ag
    team_stats[r.away]['ga'] += r.hg

    if r.hg > r.ag:
        team_stats[r.home]['wins'] += 1
        team_stats[r.away]['losses'] += 1
    elif r.hg < r.ag:
        team_stats[r.away]['wins'] += 1
        team_stats[r.home]['losses'] += 1
    else:
        team_stats[r.home]['draws'] += 1
        team_stats[r.away]['draws'] += 1

results_df = pd.DataFrame({
    'Team': teams,
    'P': [team_stats[t]['played'] for t in teams],
    'W': [team_stats[t]['wins'] for t in teams],
    'D': [team_stats[t]['draws'] for t in teams],
    'L': [team_stats[t]['losses'] for t in teams],
    'GF': [team_stats[t]['gf'] for t in teams],
    'GA': [team_stats[t]['ga'] for t in teams],
    'GD': [team_stats[t]['gf'] - team_stats[t]['ga'] for t in teams],
    'Curr': [points[t] for t in teams],
    'Proj': [avg_points[t] for t in teams],
    'Title%': [title_wins[t] / n_sims * 100 for t in teams]
}).sort_values('Proj', ascending=False).reset_index(drop=True)

results_df.index = results_df.index + 1

print("\n" + "="*110)
print("📊 PREDICTED FINAL PREMIER LEAGUE TABLE 2025/26")
print("="*110)
print(results_df.to_string(index=True, float_format=lambda x: f'{x:.1f}'))


📊 PREDICTED FINAL PREMIER LEAGUE TABLE 2025/26
                          Team   P   W  D   L   GF   GA    GD  Curr  Proj  Title%
1                   Arsenal FC  21  15  4   2 40.0 14.0  26.0    49  81.2    80.4
2           Manchester City FC  21  13  4   4 45.0 19.0  26.0    43  72.9    12.1
3               Aston Villa FC  21  13  4   4 33.0 24.0   9.0    43  71.7     7.1
4                 Liverpool FC  21  10  5   6 32.0 28.0   4.0    35  65.9     0.4
5                   Chelsea FC  21   8  7   6 34.0 24.0  10.0    31  61.6     0.0
6                 Brentford FC  21  10  3   8 35.0 28.0   7.0    33  61.3     0.0
7          Newcastle United FC  21   9  5   7 32.0 27.0   5.0    32  61.1     0.0
8            Crystal Palace FC  21   7  7   7 22.0 23.0  -1.0    28  60.7     0.0
9                    Fulham FC  21   9  4   8 30.0 30.0   0.0    31  60.5     0.0
10                  Everton FC  21   8  5   8 23.0 25.0  -2.0    29  57.7     0.0
11              Sunderland AFC  21   7  9   5 21.0